# Spaceship Titanic
## 목표
캐글의  [Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic/submissions) 데이터를 이용하여 우주선 타이타닉이 시공간 이상과 충돌하는 동안 승객이 다른 차원으로 이동되었는지 여부를 예측
## 데이터
데이터는 약 8700개의 훈련 데이터와 약 4300개의 훈련 데이터를 가지고 있으며 14개의 행으로 이루어져 있다.

 - PassengerId: 승객의 고유 ID
 - HomePlanet: 승객이 출발한 행성
 - CryoSleep: 냉동 수면 상태
 - Cabin: 승객이 머물고 있는 객실 번호 (deck/num/side형태)
 - Destination: 도착지
 - Age: 승객의 나이
 - VIP: VIP 서비스 지불 여부
 - RoomService, FoodCourt, ShoppingMall, Spa, VRDeck: 우주선 타이타닉의 고급 편의 시설 사용 비용
 - Name: 승객의 이름
 - Transported: 다른 차원으로 이동 여부 (예측 대상)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## 데이터 전처리
 - 삭제한 데이터: Name
 - 추가한 데이터: Side, Desk (Cabin의 deck, side 추출)
 - 결측치: 숫자형 데이터의 경우 중앙값으로 대체 후 표준화 하였고, 그 이외의 데이터는 최빈값으로 대체한 후 원핫인코딩 하였다.

In [ ]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")

data_train = df_train.drop(['Name'], axis =1)
data_test = df_test.drop(['Name'], axis =1)

data_train['Side']=data_train["Cabin"].str[-1]
data_train['Desk']=data_train["Cabin"].str[0]

data_test['Side']=data_test["Cabin"].str[-1]
data_test['Desk']=data_test["Cabin"].str[0]

In [ ]:
X = data_train.drop(['Transported'],axis = 1)
y = data_train.Transported.astype('int')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, test_size=0.2,random_state=42)
X_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Side,Desk
3600,3868_04,Earth,False,G/630/S,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,S,G
1262,1336_01,Earth,True,G/201/S,TRAPPIST-1e,17.0,False,0.0,0.0,0.0,0.0,0.0,S,G
8612,9193_01,Earth,NaN,G/1483/S,PSO J318.5-22,35.0,False,0.0,0.0,0.0,0.0,0.0,S,G
5075,5422_01,Europa,True,D/164/S,55 Cancri e,26.0,False,0.0,0.0,0.0,NaN,0.0,S,D
4758,5076_01,Earth,False,G/818/P,TRAPPIST-1e,13.0,False,0.0,0.0,60.0,1.0,5147.0,P,G


## 모델 및 결과
 모델은 사이킷런의 SVM을 사용하였고 또한 GridSearchCV를 사용하여 SVM의 최적의 파라미터를 찾았다. 그 결과 검증 데이터의 정확도는 0.8090856814261069, 테스트 데이터의 정확도는  0.79962, 최적의 파라미터는 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'} 라는 결과를 얻을 수 있었다.

In [ ]:
cat_cols = [col for col in X_train.columns if X_train[col].dtypes != 'float64']
num_cols = [col for col in X_train.columns if X_train[col].dtypes == 'float64']

In [ ]:
# 결측치 처리
num_impute = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler',StandardScaler())
])

cat_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_impute, num_cols),
        ('cat', cat_transform, cat_cols)
    ])

In [ ]:
# 그리드 서치cv 및 훈련
param_range =[0.001, 0.01, 0.1, 1, 10, 100]
param_grid = [{'C': param_range, 
               'kernel': ['linear']},
              {'C': param_range, 
               'gamma': param_range, 
               'kernel': ['rbf']}]
model = SVC()
grid_search = GridSearchCV(model, param_grid, return_train_score=True)

result_model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', grid_search)
                     ])


result_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                     

In [ ]:
# 검증
pred = result_model.predict(X_val)
print('Accuracy: ', accuracy_score(y_val, pred))
print('best params: ', grid_search.best_params_)

Accuracy:  0.8090856814261069
best params:  {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
